<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Set Up
%%capture
!git clone https://github.com/GenaroHacker/creating_chord_collection.git

In [2]:
# @title Chord
class GuitarChord:
    all_notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    open_string_notes = ["E", "B", "G", "D", "A", "E"]

    def __init__(self, root, chord_type, starting_fret, difficulty, finger_ascending):
        self.root = root
        self.chord_type = chord_type
        self.starting_fret = starting_fret
        self.difficulty = difficulty
        self.finger_ascending = finger_ascending

    def __str__(self):
        return f"Guitar Chord: {self.root}{self.chord_type}, Starting Fret: {self.starting_fret}, Difficulty: {self.difficulty}, Finger Positions: {self.finger_ascending}"

    def get_string_notes(self):
        starting_fret = self.starting_fret if self.starting_fret is not None else 0

        string_notes = []
        for string, finger_position in zip(GuitarChord.open_string_notes, self.finger_ascending):
            if finger_position is None:
                # If finger position is None, the string is not played
                string_notes.append(None)
            else:
                # Calculate the note index, considering the starting fret
                note_index = (GuitarChord.all_notes.index(string) + finger_position + starting_fret) % len(GuitarChord.all_notes)
                string_notes.append(GuitarChord.all_notes[note_index])

        return string_notes

    def is_open(self):
        return 0 in self.finger_ascending

    def get_absolute_notes(self):
        string_notes = self.get_string_notes()
        filtered_notes = filter(None, string_notes)  # Remove None values
        unique_notes = list(dict.fromkeys(filtered_notes))  # Remove duplicates

        # Sort the notes as per their order in all_notes
        sorted_notes = sorted(unique_notes, key=lambda x: GuitarChord.all_notes.index(x))
        return sorted_notes

# Example usage
chord = GuitarChord('E', 'm', 0, 1, [0, 0, 0, 2, 2, 0])
print(chord)
print("String Notes:", chord.get_string_notes())
print("Is Open Chord:", chord.is_open())
print("Absolute Notes:", chord.get_absolute_notes())



Guitar Chord: Em, Starting Fret: 0, Difficulty: 1, Finger Positions: [0, 0, 0, 2, 2, 0]
String Notes: ['E', 'B', 'G', 'E', 'B', 'E']
Is Open Chord: True
Absolute Notes: ['E', 'G', 'B']


In [3]:
# @title Collection
import sqlite3
from collections import defaultdict

class ChordCollection:
    def __init__(self):
        self.chords = defaultdict(list)

    def load(self, db_path):
        self.chords.clear()
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()

        cursor.execute('SELECT ROOT, TYPE, DIFFICULTY, STARTING_FRET, STRING_1, STRING_2, STRING_3, STRING_4, STRING_5, STRING_6 FROM TABLE_CHORDS')
        rows = cursor.fetchall()

        for row in rows:
            root, chord_type, difficulty, starting_fret, *fingers = row
            chord = GuitarChord(root, chord_type, starting_fret, difficulty, fingers)
            self.chords[root].append(chord)

        connection.close()

# Example usage
chord_collection = ChordCollection()
chord_collection.load('/content/creating_chord_collection/chord_collection.db')
chord = chord_collection.chords["E"][6]
print(chord)
print("String Notes:", chord.get_string_notes())
print("Is Open Chord:", chord.is_open())
print("Absolute Notes:", chord.get_absolute_notes())


Guitar Chord: Em, Starting Fret: None, Difficulty: 3, Finger Positions: [0, 0, 0, 2, 2, 0]
String Notes: ['E', 'B', 'G', 'E', 'B', 'E']
Is Open Chord: True
Absolute Notes: ['E', 'G', 'B']
